## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-08-23-50-00 +0000,wsj,Perky Australian Dollar Looks to Extend Gains,https://www.wsj.com/finance/currencies/perky-a...
1,2025-12-08-23-49-37 +0000,nypost,Man stabbed by illegal migrant on NC train was...,https://nypost.com/2025/12/08/us-news/kenyon-d...
2,2025-12-08-23-43-06 +0000,nypost,Zelensky says he still has one big unanswered ...,https://nypost.com/2025/12/08/world-news/zelen...
3,2025-12-08-23-39-55 +0000,nyt,Marooned Off the English Coast: Lots of Bananas,https://www.nytimes.com/2025/12/08/world/europ...
4,2025-12-08-23-37-00 +0000,bbc,Thieves snatch eight Matisse artworks from lib...,https://www.bbc.com/news/articles/c0mp4nevdz7o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2352/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
83,trump,55
144,warner,21
27,deal,17
353,court,15
145,netflix,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
201,2025-12-08-15-28-04 +0000,nypost,Paramount Skydance launches hostile bid for Wa...,https://nypost.com/2025/12/08/business/paramou...,179
215,2025-12-08-14-53-40 +0000,latimes,Paramount launches hostile $78-billion bid for...,https://www.latimes.com/business/story/2025-12...,151
223,2025-12-08-14-06-59 +0000,nyt,Trump Looms Over Netflix’s Deal for Warner Bro...,https://www.nytimes.com/2025/12/08/business/de...,131
253,2025-12-08-10-44-44 +0000,bbc,Trump says $72bn Netflix-Warner Bros deal 'cou...,https://www.bbc.com/news/articles/cn815egjqjpo...,129
302,2025-12-08-02-40-46 +0000,wapo,Trump says he will be ‘involved’ in vetting Ne...,https://www.washingtonpost.com/business/2025/1...,126


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
201,179,2025-12-08-15-28-04 +0000,nypost,Paramount Skydance launches hostile bid for Wa...,https://nypost.com/2025/12/08/business/paramou...
53,60,2025-12-08-22-17-29 +0000,bbc,Alina Habba resigns after court disqualified h...,https://www.bbc.com/news/articles/cj69x814dkyo...
308,57,2025-12-08-01-54-25 +0000,nypost,JPMorgan Chase’s Jamie Dimon claims AI will no...,https://nypost.com/2025/12/07/business/jamie-d...
229,50,2025-12-08-13-20-35 +0000,nypost,‘Disappointed’ Trump claims Zelensky ‘hasn’t y...,https://nypost.com/2025/12/08/world-news/disap...
133,46,2025-12-08-19-23-51 +0000,nypost,China’s trade surplus hits $1 trillion for fir...,https://nypost.com/2025/12/08/business/chinas-...
90,44,2025-12-08-21-09-00 +0000,wsj,Supreme Court Appears Ready to Give President ...,https://www.wsj.com/politics/policy/supreme-co...
272,40,2025-12-08-09-28-00 +0000,wsj,Japan Is Out Spending. Bond Markets Seem Nervo...,https://www.wsj.com/economy/japan-is-out-spend...
17,37,2025-12-08-23-05-39 +0000,nyt,Trump Plans $12 Billion Bailout to Aid Farmers...,https://www.nytimes.com/2025/12/08/us/politics...
100,33,2025-12-08-20-43-55 +0000,nypost,Israeli foreign minister laughs off Zohran Mam...,https://nypost.com/2025/12/08/us-news/israeli-...
304,31,2025-12-08-02-19-24 +0000,bbc,Former Australian deputy PM joins anti-immigra...,https://www.bbc.com/news/articles/cqxq8g4d873o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
